In [3]:
import dev.langchain4j.agent.tool.Tool
import dev.langchain4j.model.chat.ChatLanguageModel
import dev.langchain4j.model.chat.StreamingChatLanguageModel
import dev.langchain4j.model.ollama.OllamaChatModel
import dev.langchain4j.model.ollama.OllamaStreamingChatModel
import dev.langchain4j.service.AiServices
import dev.langchain4j.service.SystemMessage
import dev.langchain4j.service.Result
import kotlinx.coroutines.runBlocking

val chat: ChatLanguageModel =
    OllamaChatModel.builder()
        .baseUrl("http://localhost:11434")
        .logRequests(true)
        .logResponses(true)
        .modelName("qwq")
        .temperature(0.7)
        .build()

val streaming: StreamingChatLanguageModel =
    OllamaStreamingChatModel.builder()
        .baseUrl("http://localhost:11434")
        .logRequests(true)
        .logResponses(true)
        .modelName("qwq")
        .temperature(0.7)
        .build()

enum class Currency { USD, EUR, BGP; }
data class Destination(val price: Double, val currency: Currency, val date: String)

enum class CarType { SUV; }

// value classes don't work with reflection
@JvmInline
value class BookingId(val value: String)

object Manager {

    val flights = mapOf("Paris" to Destination(700.0, Currency.USD, date = "TBD"))
    val bookings = listOf("12345")
    val cars = mapOf(CarType.SUV to listOf("VW_TOURAN"))
    val conversionRates = mapOf(
        Pair(Currency.USD, Currency.EUR) to 0.93,
        Pair(Currency.USD, Currency.BGP) to 0.81,
        Pair(Currency.EUR, Currency.USD) to 1.07,
        Pair(Currency.EUR, Currency.BGP) to 0.87,
        Pair(Currency.BGP, Currency.USD) to 1.23,
        Pair(Currency.BGP, Currency.EUR) to 1.15
    )

    @Tool("Cancel a booking")
    fun cancelBooking(bookingId: Int): Unit {
        println("Cancelled booking $bookingId")
        requireNotNull(bookings[bookingId]) { "Booking doesn't exist!" }
    }

    @Tool("Convert 'from' a certain 'originalPrice' 'to' another currency")
    fun convertPrice(originalPrice: Double, from: Currency, to: Currency): Double {
        val rate = requireNotNull(conversionRates[Pair(from, to)])
        println("Converting $from to $to with $rate")
        return originalPrice * rate
    }

    @Tool("find available flights")
    fun search(location: String): Destination? {
        println("searching location $location")
        return flights[location]
    }

    @Tool("Search available rental cars")
    fun rentals(type: CarType): List<String>? {
        println("Searching rentals")
        return cars[type]
    }
}

interface MathGenius {
    // TODO Notebook doesn't pick up TokenStreamToStringFlowAdapter through META-INF.services
    @SystemMessage("You're a booking manager that follows users orders")
    fun ask(question: String): Result<String>
}

val genius = AiServices.builder(MathGenius::class.java)
    .streamingChatLanguageModel(streaming)
    .chatLanguageModel(chat)
    .tools(Manager)
    .build()

runBlocking {
    val result =
        genius.ask(
            """
            Can you cancel my booking with id 12345?
            Find me a new available flight to Paris?
            Convert the price of the flight to EURO.
            And finally, Find me a rental large car when I arrive.
            """
        )
    println(result.content())
    println(result.toolExecutions())
    result.toolExecutions()
        .map { it.request().name() }
        .containsAll(listOf("cancelBooking", "search", "rentals"))
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[3], line 90, column 1: Class 'kotlinx.coroutines.BuildersKt__BuildersKt' was compiled with an incompatible version of Kotlin. The actual metadata version is 2.1.0, but the compiler version 1.9.0 can read versions up to 2.0.0.
The class is loaded from /Users/simonvergauwen/.gradle/caches/modules-2/files-2.1/org.jetbrains.kotlinx/kotlinx-coroutines-core-jvm/1.10.1/fe066928754beda3d59c8282e04289546465a360/kotlinx-coroutines-core-jvm-1.10.1.jar!/kotlinx/coroutines/BuildersKt__BuildersKt.class